In [ ]:
import numpy as np

In [ ]:
# create a binomial lattice with dividends 
def binomial_lattice_div(n, sigma, r, T, S0, K, D0, p, div, payoff_func, div_payoff_func):
    # sort the div array in descending order
    div = np.sort(div)
    div = np.ceil(div * n).astype(int)
    div = div[::-1]

    # create the arrays for the stock and option values
    stock_array = np.zeros((n+1))
    new_stock_array = np.zeros((n+1))
    option_array = np.zeros((n+1))
    new_option_array = np.zeros((n+1))

    # u and d
    u = np.exp(sigma*np.sqrt(T/n) + (r - 0.5*(sigma**2))*T/n)
    d = np.exp(-sigma*np.sqrt(T/n) + (r - 0.5*(sigma**2))*T/n)
    q = 1/2

    # Time N
    for j in range(0,n+1):
        # set the element to the sum of the two elements above it
        stock_array[j] = S0 * (u**j) * (d**(n-j))
        option_array[j] = payoff_func(stock_array[j], K, sigma, n, T)

    # loop through the lattice backwards
    for i in range(n, 0, -1):
        # check if the current time is a dividend time
        if (i in div):
            # interpolate the option array to the stock array for the dividend
            option_array = np.interp(stock_array - div_payoff_func(stock_array, D0, p), stock_array, option_array).copy()
            # display(pd.DataFrame({'Div Stock array ' + str(i) : stock_array - div_payoff_func(stock_array, D0, p), 'Div Option array ' + str(i) : option_array}))

        # get lagged stock and option array for calculation
        stock_array_r = np.roll(stock_array, -1)
        option_array_r = np.roll(option_array, -1)
        # set the last value to nan
        option_array_r[-1] = np.nan
        stock_array_r[-1] = np.nan
        # calculate the new stock and option array
        new_stock_array = np.exp(-r * T / n) * q * (stock_array_r + stock_array)
        new_option_array = np.exp(-r * T / n) * q * (option_array_r + option_array)
        # set the arrays to the new arrays
        stock_array = new_stock_array.copy()
        option_array = new_option_array.copy()
        # reset the new row
        new_stock_array = np.zeros((n+1))
        new_option_array = np.zeros((n+1))

    # return the array
    return option_array